In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
from pprint import pprint
import numpy as np


sst_dataset = load_dataset('sst')
pprint(sst_dataset['train'][0])
print(np.shape(sst_dataset))


In [ ]:
pprint(sst_dataset['train'][0]['sentence'].split(' '))
#print(len(sst_dataset['train']))
new_vocab=['PAD', 'UNK']                    #없어도 되긴함
for i in range(len(sst_dataset['train'])):
  new_vocab+=list(sst_dataset['train'][i]['sentence'].split(' '))

vocab_count={}
for voca in new_vocab:
  try:
    vocab_count[voca]+=1
  except:
    vocab_count[voca]=1  
#print(vocab_count)

new_vocab2=['PAD','UNK']
for key,value in vocab_count.items():
  if (vocab_count[key]>=2):
    new_vocab2.append(key)
    #print(key)

print(new_vocab2)
print(len(new_vocab2))
#print(len(new_vocab))                       #이게 정답
#print(new_vocab)  

new_word2id={word:id for id,word in enumerate(new_vocab2)}
print(new_word2id['he'])

In [ ]:
# Two-layer MLP classification
print(len(new_vocab2))
class Baseline(nn.Module):
  def __init__(self, d, length):
    super(Baseline, self).__init__()
    self.embedding = nn.Embedding(len(new_vocab2), d)   #임베딩테이블을 만듦. 행은 vocab 인덱스,열은 그것의 벡터를 저장 ex 1행엔 1번 vocab의 벡터가 저장 
    self.layer = nn.Linear(d * length, d, bias=True)
    self.relu = nn.ReLU()
    self.class_layer = nn.Linear(d, 2, bias=True)

  def forward(self, input_tensor):
    emb = self.embedding(input_tensor) # [batch_size, length, d]          #https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html self.embedding(input) input의 1,3,0,,,마다 word embedding값을 output으로 함. 1,3..은 각각 3개짜리 벡터로 변환 
    #print(self.embedding.weight.shape)
    print(emb)
    emb_flat = emb.view(emb.size(0), -1) # [batch_size, length*d]
    hidden = self.relu(self.layer(emb_flat))
    logits = self.class_layer(hidden)
    return logits
print(vocab)
d = 3 # usually bigger, e.g. 128
baseline = Baseline(d, length)
logits = baseline(input_tensor)
softmax = nn.Softmax(1)
print(softmax(logits)) # probability for each class

cel = nn.CrossEntropyLoss()
label = torch.LongTensor([1]) # The ground truth label for "hi world!" is positive.
print(label)
loss = cel(logits, label) # Loss, a.k.a L
print(loss)

optimizer = torch.optim.SGD(baseline.parameters(), lr=0.1)
optimizer.zero_grad() # reset process
loss.backward() # compute gradients
optimizer.step() # update parameters

In [ ]:
#data preprocessing


import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from torch import nn
baseline = Baseline(64, 16)
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
baseline=baseline.to(dev)
inputs, labels = [], []
length = 16
input_ = sst_dataset['train']['sentence']
print(len(input_))
print(len(new_vocab2))
print(input_[0])
for i in range(len(input_)):

  input_tokens = input_[i].split(' ')

  input_ids = [new_word2id[word] if word in new_word2id else 1 for word in input_tokens] # UNK if word not found
  
  if len(input_ids) < length:
    input_ids = input_ids + [0] * (length - len(input_ids)) # PAD tokens at the end
  else:
    input_ids = input_ids[:length]
  inputs.append(input_ids)
  
  
  label=(sst_dataset['train']['label'][i]>0.5)
  labels.append(label)

input_tensor = torch.LongTensor([inputs]) # the first dimension is minibatch size
labels_tensor=torch.LongTensor([labels])
input_tensor=input_tensor.reshape(-1,16)
labels_tensor=labels_tensor.reshape(-1,1)
print(input_tensor)
print(labels_tensor)

test_inputs, test_labels = [], []
test_input_ = sst_dataset['validation']['sentence']
print(len(test_input_))
print(len(new_vocab2))
print(test_input_[0])
for i in range(len(test_input_)):

  test_input_tokens = test_input_[i].split(' ')

  test_input_ids = [new_word2id[word] if word in new_word2id else 1 for word in test_input_tokens] # UNK if word not found
  
  if len(test_input_ids) < length:
    test_input_ids = test_input_ids + [0] * (length - len(test_input_ids)) # PAD tokens at the end
  else:
    test_input_ids = test_input_ids[:length]
  test_inputs.append(test_input_ids)
  
  
  test_label=(sst_dataset['validation']['label'][i]>0.5)
  test_labels.append(test_label)

test_input_tensor = torch.LongTensor([test_inputs]) # the first dimension is minibatch size
test_labels_tensor=torch.LongTensor([test_labels])
test_input_tensor=test_input_tensor.reshape(-1,16)
test_labels_tensor=test_labels_tensor.reshape(-1,1)
print(test_input_tensor.shape)
print(test_labels_tensor.shape)







In [ ]:
#3.1 

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

dataset = TensorDataset(input_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
baseline = Baseline(64,16)
baseline=baseline.to(dev)
cel = nn.CrossEntropyLoss()
  # We use Adam optimizer. This allows us to not worry about learning rate
optimizer = torch.optim.Adam(baseline.parameters(), lr=0.0001)
epochs = 10
for epoch in range(epochs):
  for batch_idx, samples in enumerate(dataloader):
    
    x_train, y_train = samples[0].to(dev), samples[1].to(dev)
    #print(samples[0].shape)
    #print(samples[1].shape)
    
    optimizer.zero_grad()
    outputs=baseline(x_train)
    print('output',outputs)
    y_train=y_train[:,0]                                    #차원축소 해줘야함
    loss=cel(outputs,y_train)
    print('LOSS:',loss)


    loss.backward()
    optimizer.step()


testset = TensorDataset(test_input_tensor, test_labels_tensor)
test_dataloader = DataLoader(testset, batch_size=16)
correct_count=0
total_count=0
with torch.no_grad():
    for data in test_dataloader:
        x_test,y_test = data[0].to(dev), data[1].to(dev)
        test_outputs = baseline(x_test)
        _, preds = test_outputs.max(1)
        y_test=y_test[:,0]   #행을 기준으로 큰 값을 갖는 인덱스 반환
        print('y_test:',y_test)
        print('preds',preds)
        num_correct = (y_test == preds).long().sum()
        total_count += x_test.size(0)
        correct_count += num_correct
        
        loss = cel(test_outputs, y_test)
        print(loss)
        print('correct count: ',num_correct)
        print('size:',x_test.size(0)) 

        
    #print('correct count: ',correct_count)
    #print('total:',total_count)    
    print("accuracy:",correct_count/total_count)       
   

RNN

In [ ]:
#3.2 RNN

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size,output_size):           #input은 (i,j) (16,64)로 들어온다. i번째 글자의 임베딩벡터                      #input이 3차원으로 들어오면 (batch, sequence length, embedding d) 여기선 (16,16,64)
    super(RNN, self).__init__()
    self.input_linear = nn.Linear(input_size,hidden_size,bias=True)   
    self.hidden_linear = nn.Linear(hidden_size,hidden_size, bias=True)
    self.tanh = nn.Tanh()
    

  def forward(self, input_tensor,hidden):
    hidden=self.tanh(self.input_linear(input_tensor)+self.hidden_linear(hidden))

    return hidden


class RNNlayer(nn.Module):
  def __init__(self,module):
    super(RNNlayer,self).__init__()
    self.rnn=module
    self.out_size=64


  def forward(self,input_tensor):
  
    cur_hidden=torch.zeros(input_tensor.shape[0],input_tensor.shape[2])        #input이 3차원으로 들어오면 (batch, sequence length, embedding d) 여기선 (16,16,64)
    if input_tensor.is_cuda:
      cur_hidden = cur_hidden.cuda()


    for i in range(input_tensor.shape[1]):                                      #input이 2차원으로 줄어든다.
      cur_input=input_tensor[:,i,:]
      cur_hidden=self.rnn(cur_input,cur_hidden)
    return cur_hidden
    

class Classification(nn.Module):
  def __init__(self,module,d):
    super(Classification,self).__init__()
    self.d=d
    self.linear=nn.Linear(d,2,bias=True)
    self.embedding = nn.Embedding(len(new_vocab2), d)
    self.layer=module

  def forward(self,input_tensor):
    emb = self.embedding(input_tensor)
    outputs=self.layer(emb)
    outputs=self.linear(emb)
    return outputs


rnn=RNN(64,64,64)
rnn_layer=RNNlayer(rnn)
classification=Classification(rnn_layer,64)

dataset = TensorDataset(input_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
 
classification = classification.cuda()
cel = nn.CrossEntropyLoss()
  # We use Adam optimizer. This allows us to not worry about learning rate
optimizer = torch.optim.Adam(baseline.parameters(), lr=0.0001)
epochs = 10
for epoch in range(epochs):
  for batch_idx, samples in enumerate(dataloader):
    
    x_train, y_train = samples[0].to(dev), samples[1].to(dev)
    #print(samples[0].shape)
    #print(samples[1].shape)
    
    optimizer.zero_grad()
    outputs=classification(x_train)
    outputs=outputs[:,0] 
    print(outputs)
    y_train=y_train[:,0] 
    print(y_train)                                   #차원축소 해줘야함
    loss=cel(outputs,y_train)
    print('LOSS:',loss)


    loss.backward()
    optimizer.step()


testset = TensorDataset(test_input_tensor, test_labels_tensor)
test_dataloader = DataLoader(testset, batch_size=16)
correct_count=0
total_count=0
with torch.no_grad():
    for data in test_dataloader:
        x_test,y_test = data[0].to(dev), data[1].to(dev)
        test_outputs = classification(x_test)
        test_outputs=test_outputs[:,0] 
        _, preds = test_outputs.max(1)
        y_test=y_test[:,0]   #행을 기준으로 큰 값을 갖는 인덱스 반환
        print('y_test:',y_test)
        print('preds',preds)
        num_correct = (y_test == preds).long().sum()
        total_count += x_test.size(0)
        correct_count += num_correct
        
        loss = cel(test_outputs, y_test)
        print(loss)
        print('correct count: ',num_correct)
        print('size:',x_test.size(0)) 

        
    #print('correct count: ',correct_count)
    #print('total:',total_count)    
    print("accuracy:",correct_count/total_count)       
   


LSTM

In [ ]:


class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size,output_size):           #input, hidden은 (i,j) (16,64)로 들어온다. i번째 글자의 임베딩벡터                      #input이 3차원으로 들어오면 (batch, sequence length, embedding d) 여기선 (16,16,64)
    super(LSTM, self).__init__()
    self.FG_input_linear = nn.Linear(input_size,hidden_size,bias=True)   
    self.FG_hidden_linear = nn.Linear(hidden_size,hidden_size, bias=True)

    self.IG_input_linear = nn.Linear(input_size,hidden_size,bias=True)   
    self.IG_hidden_linear = nn.Linear(hidden_size,hidden_size, bias=True)

    self.OG_input_linear = nn.Linear(input_size,hidden_size,bias=True)   
    self.OG_hidden_linear = nn.Linear(hidden_size,hidden_size, bias=True)

    self.CS_input_linear = nn.Linear(input_size,hidden_size,bias=True)   
    self.CS_hidden_linear = nn.Linear(hidden_size,hidden_size, bias=True)


    self.tanh = nn.Tanh()
    self.sigmoid=nn.Sigmoid()
    

  def forward(self, input_tensor,hidden,prev_cell_state):
    forget=self.sigmoid(self.FG_input_linear(input_tensor)+self.FG_hidden_linear(hidden))

    input=self.sigmoid(self.IG_input_linear(input_tensor)+self.IG_hidden_linear(hidden))

    output=self.sigmoid(self.OG_input_linear(input_tensor)+self.OG_hidden_linear(hidden))

    cell_state=self.tanh(self.CS_input_linear(input_tensor)+self.CS_hidden_linear(hidden))

    cell_state=prev_cell_state*forget+input*cell_state

    hidden=output*self.tanh(cell_state)

    return hidden,cell_state


class LSTMlayer(nn.Module):
  def __init__(self,module):
    super(LSTMlayer,self).__init__()
    self.lstm=module
    self.out_size=64
    

  def forward(self,input_tensor):
  
    cur_hidden=torch.zeros(input_tensor.shape[0],input_tensor.shape[2])        #input이 3차원으로 들어오면 (batch, sequence length, embedding d) 여기선 (16,16,64)
    cur_state=torch.zeros(input_tensor.shape[0],input_tensor.shape[2])  
   
    #if input_tensor.is_cuda:
      #cur_hidden_tensor = cur_hidden_tensor.cuda()


    for i in range(input_tensor.shape[1]):                                      #input이 2차원으로 줄어든다.

      cur_input=input_tensor[:,i,:]
      cur_hidden,cur_state=self.lstm(cur_input,cur_hidden,cur_state)

    return cur_hidden





class LSTMClassification(nn.Module):
  def __init__(self,module,d):
    super(LSTMClassification,self).__init__()
    self.d=d
    self.linear=nn.Linear(d,2,bias=True)
    self.embedding = nn.Embedding(len(new_vocab2), d)
    self.layer=module
    

  def forward(self,input_tensor):
    emb = self.embedding(input_tensor)
    outputs=self.layer(emb)
    
    
    outputs=self.linear(emb)
    return outputs


lstm=LSTM(64,64,64)
lstm_layer=LSTMlayer(lstm)

lstmclassification=LSTMClassification(lstm_layer,64)

dataset = TensorDataset(input_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

cel = nn.CrossEntropyLoss()
  # We use Adam optimizer. This allows us to not worry about learning rate
optimizer = torch.optim.Adam(baseline.parameters(), lr=0.0001)
epochs = 10
for epoch in range(epochs):
  for batch_idx, samples in enumerate(dataloader):
    
    x_train, y_train = samples[0], samples[1]
    #print(samples[0].shape)
    #print(samples[1].shape)
    
    optimizer.zero_grad()
    outputs=lstmclassification(x_train)
    outputs=outputs[:,0] 
    print(outputs)
    y_train=y_train[:,0] 
    print(y_train)                                   #차원축소 해줘야함
    loss=cel(outputs,y_train)
    print('LOSS:',loss)


    loss.backward()
    optimizer.step()


testset = TensorDataset(test_input_tensor, test_labels_tensor)
test_dataloader = DataLoader(testset, batch_size=16)
correct_count=0
total_count=0
with torch.no_grad():
    for data in test_dataloader:
        x_test,y_test = data[0], data[1]
        test_outputs = lstmclassification(x_test)
        test_outputs=test_outputs[:,0] 
        _, preds = test_outputs.max(1)
        y_test=y_test[:,0]   #행을 기준으로 큰 값을 갖는 인덱스 반환
        print('y_test:',y_test)
        print('preds',preds)
        num_correct = (y_test == preds).long().sum()
        total_count += x_test.size(0)
        correct_count += num_correct
        
        loss = cel(test_outputs, y_test)
        print(loss)
        print('correct count: ',num_correct)
        print('size:',x_test.size(0)) 

        
    #print('correct count: ',correct_count)
    #print('total:',total_count)    
    print("accuracy:",correct_count/total_count)     